# Data Cleaning and Munging Script - City Data Portal

In [171]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
pd.options.mode.chained_assignment = None
sns.set()

In [16]:
austin_zips = (78701,
78702,
78703,
78704,
78705,
78717,
78719,
78721,
78722,
78723,
78724,
78725,
78726,
78727,
78728,
78729,
78730,
78731,
78732,
78733,
78734,
78735,
78736,
78737,
78738,
78739,
78741,
78742,
78744,
78745,
78746,
78747,
78748,
78749,
78750,
78751,
78752,
78753,
78754,
78756,
78757,
78758,
78759,
78617,
78653,
78660)

### Permits Data

In [4]:
permits_data = pd.read_csv("raw_data/Issued_Construction_Permits.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (54,58,59,60,61,62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [90]:
permits = permits_data.loc[:,('Permit Num', 'Permit Class Mapped', 
                              'Calendar Year Issued', 'Original Zip')]

permits = permits[permits['Calendar Year Issued'] >= 2011]
permits = permits[permits['Original Zip'].isin(austin_zips)]

res_permits = permits[permits['Permit Class Mapped'] == "Residential"].drop('Permit Class Mapped', axis = 1)
com_permits = permits[permits['Permit Class Mapped'] == "Commercial"].drop('Permit Class Mapped', axis = 1)

In [311]:
res_permits_year_count = res_permits.groupby(['Calendar Year Issued', 'Original Zip']).size().reset_index()
res_permits_year_count.rename(columns = {0: "count", 'Original Zip': 'zip_code', 'Calendar Year Issued': 'year'}, inplace = True)
res_permits_year_count['var_type'] = 'res_permit_count'

In [229]:
com_permits_year_count = com_permits.groupby(['Calendar Year Issued', 'Original Zip']).size().reset_index()
com_permits_year_count.rename(columns = {0: "count", 'Original Zip': 'zip_code', 'Calendar Year Issued': 'year'}, inplace = True)
com_permits_year_count['var_type'] = 'commercial_permit_count'

In [312]:
res_permits_year_count.rename(columns = {'count': 'res_permit_count'}, inplace = True)
res = res_permits_year_count.drop('var_type', axis = 1)

In [304]:
com_permits_year_count.rename(columns = {'count': 'com_permit_count'}, inplace = True)
com = com_permits_year_count.drop('var_type', axis = 1)

In [314]:
all_var_df = pd.merge(com, res, on = ('year', 'zip_code'), how = 'outer')

### Crime Data

In [158]:
crime_data = pd.read_csv("raw_data/Crime_Reports.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [240]:
crime = crime_data.loc[:,('Incident Number', 'Highest Offense Description', 'Occurred Date Time', 'Zip Code')]

In [241]:
conditions = [
    (crime['Highest Offense Description'].isin(['ASSAULT BY CONTACT', 'ASSAULT WITH INJURY', 'ASSAULT W/INJURY-FAM/DATE VIOL'])),
    (crime['Highest Offense Description'].isin(['AUTO-THEFT', 'BURGLARY NON RESIDENCE', 'BURGLARY OF RESIDENCE', 'BURGLARY OF VEHICLE', 'THEFT', 'THEFT BY SHOPLIFTING'])),
    (crime['Highest Offense Description'].isin(['CRIMINAL MISCHIEF', 'CRIMINAL TRESPASS', 'DISTURBANCE - OTHER', 'FAMILY DISTURBANCE', 'HARASSMENT'])),
    (crime['Highest Offense Description'].isin(['CUSTODY ARREST TRAFFIC WARR', 'WARRANT ARREST NON TRAFFIC'])),
    (crime['Highest Offense Description'].isin(['DWI', 'PUBLIC INTOXICATION'])),
    (crime['Highest Offense Description'].isin(['IDENTITY THEFT'])),
    (crime['Highest Offense Description'].isin(['POSS CONTROLLED SUB/NARCOTIC', 'POSS OF DRUG PARAPHERNALIA', 'POSSESSION OF MARIJUANA'])),
    (crime['Highest Offense Description'].isin(['RUNAWAY CHILD'])),
    ]
     
values = ['assault', 'theft', 'criminal_activity', 'warrent_arrest', 
          'public_intoxication', 'identity_theft', 'narcotics', 'runaway_child']

crime['Offense Category'] = np.select(conditions, values)
crime = crime[crime['Offense Category'] != '0'].drop('Highest Offense Description', axis = 1)

In [242]:
crime.loc[:,'Occurred Date Time'] = pd.to_datetime(crime.loc[:,'Occurred Date Time'])
crime['year'] = crime.loc[:,'Occurred Date Time'].dt.to_period("Y")
crime = crime.drop('Occurred Date Time', axis = 1)

In [358]:
crime_count = crime.groupby(['Offense Category', 'year', 'Zip Code']).size().reset_index()
crime_count['var_type'] = 'count_' + crime_count['Offense Category'] + '_crime'
crime_count.rename(columns = {'Zip Code': 'zip_code', 0:'count'}, inplace = True)
crime_count = crime_count.drop('Offense Category', axis = 1)
crime_count = crime_count[crime_count['year'] >= pd.Period('2011')]

In [409]:
reshaped_df = crime_count.pivot(index=["year", "zip_code"], columns="var_type").reset_index()

reshaped_df#.rename_axis(None, axis=1)#.reset_index(drop=True)

year zip_code               count                                \
var_type                count_assault_crime count_criminal_activity_crime   
0         2011  78610.0                 NaN                           1.0   
1         2011  78613.0                16.0                          41.0   
2         2011  78617.0                66.0                         278.0   
3         2011  78652.0                 NaN                           7.0   
4         2011  78653.0                 5.0                          28.0   
..         ...      ...                 ...                           ...   
560       2021  78754.0                27.0                          75.0   
561       2021  78756.0                11.0                          43.0   
562       2021  78757.0                30.0                          96.0   
563       2021  78758.0               147.0                         381.0   
564       2021  78759.0                35.0                         144.0   

                                                           \
var_type count_identity_theft_crime count_narcotics_crime   
0                               NaN                   NaN   
1                               NaN                   4.0   
2                              12.0                  32.0   
3                               NaN                   1.0   
4                               4.0                   2.0   
..                              ...                   ...   
560                            30.0                   7.0   
561                            14.0                   1.0   
562                            46.0                  10.0   
563                            65.0                  28.0   
564                            62.0                  10.0   

                                                                    \
var_type count_public_intoxication_crime count_runaway_child_crime   
0                                    NaN                       NaN   
1                                    6.0                       1.0   
2                                   17.0                      32.0   
3                                    5.0                       NaN   
4                                    2.0                       1.0   
..                                   ...                       ...   
560                                  8.0                       5.0   
561                                  3.0                       NaN   
562                                 17.0                       NaN   
563                                 46.0                      14.0   
564                                 11.0                       4.0   

                                                       
var_type count_theft_crime count_warrent_arrest_crime  
0                      2.0                        2.0  
1                    284.0                        7.0  
2                    184.0                       46.0  
3                      5.0                        1.0  
4                     22.0                        4.0  
..                     ...                        ...  
560                   53.0                        1.0  
561                   31.0                        NaN  
562                  138.0                        NaN  
563                  304.0                        1.0  
564                  156.0                        NaN  

[565 rows x 10 columns]

In [315]:
all_var_df

,year,zip_code,com_permit_count,res_permit_count
0,2011,78617.0,78.0,407.0
1,2011,78653.0,8.0,4.0
2,2011,78660.0,70.0,233.0
3,2011,78701.0,1372.0,55.0
4,2011,78702.0,520.0,1286.0
...,...,...,...,...
501,2021,78756.0,46.0,200.0
502,2021,78757.0,100.0,701.0
503,2021,78758.0,230.0,281.0
504,2021,78759.0,134.0,517.0


### Water Consumption Data

In [431]:
water_data_com = pd.read_csv("raw_data/Austin_Water_-_Commercial_Water_Consumption.csv")
water_data_com['var_type'] = 'commercial_water_use_gal'
water_data_res = pd.read_csv("raw_data/Austin_Water_-_Residential_Water_Consumption.csv")
water_data_res['var_type'] = 'res_water_use_gal'

water_data = water_data_res.append(water_data_com)
water_data['year'] = water_data['Year Month'].astype(str).str[0:4]

In [433]:
water_data.groupby(['year', 'Postal Code']).sum()

Year Month  Total Gallons
year Postal Code                           
2012 78613           9657912       63627400
     78617          17504979      549139000
     78640           2414478        2130800
     78652          12072390       14753500
     78653          12072390       30304900
...                      ...            ...
2020 78756          14342358      268429500
     78757          14544360      608046400
     78758          14544360     1388186600
     78759          16362405     1377663500
     78766           3636090          86400

[477 rows x 2 columns]

In [422]:
water = water_data.loc[:,('year', 'Postal Code', 'Total Gallons', 'var_type')]
water.rename(columns = {'Postal Code': 'zip_code'}, inplace = True)

In [427]:
water_data_com[water_data_com['Postal Code'] == 78766].head(210)

,Year Month,Postal Code,Customer Class,Total Gallons,var_type
208,201201,78766,Commercial,5700,commercial_water_use_gal
209,201201,78766,Irrigation - Commercial,26000,commercial_water_use_gal
419,201202,78766,Commercial,5700,commercial_water_use_gal
420,201202,78766,Irrigation - Commercial,24000,commercial_water_use_gal
629,201203,78766,Commercial,5200,commercial_water_use_gal
...,...,...,...,...,...
22263,202007,78766,Irrigation - Commercial,0,commercial_water_use_gal
22482,202008,78766,Commercial,7800,commercial_water_use_gal
22483,202008,78766,Irrigation - Commercial,0,commercial_water_use_gal
22700,202009,78766,Commercial,14000,commercial_water_use_gal


### 311 Data

In [437]:
three11_data = pd.read_csv("raw_data/311_Unified_Data_Test_Site_2019.csv")

/Users/carolinekinnen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [446]:
conditions = [
    (three11_data['SR Description'].isin(['Animal - Proper Care', 'Animal Control - Assistance Request', 'Found Animal Report - Keep', 'Injured / Sick Animal','Loose Dog','Wildlife Exposure'])),
    (three11_data['SR Description'].isin(['Austin Code - Request Code Officer'])),
    (three11_data['SR Description'].isin(['Graffiti Abatement', 'Public Health - Graffiti Abatement'])),
    (three11_data['SR Description'].isin(['Loud Commercial Music'])),
    (three11_data['SR Description'].isin(['Park Maintenance - Grounds'])),
    (three11_data['SR Description'].isin(['Debris in Street', 'Pothole Repair', 'Sign - Traffic Sign Maintenance', 'Street Light Issue- Address', 'Traffic Signal - Dig Tess Request', 'Traffic Signal - Maintenance'])),
    ]
     
values = ['animal_311', 'request_code_officer_311', 'graffiti_311',
          'loud_commercial_music_311', 'park_maintenance_311', 'road_repair_311']

three11_data['Reporting Category'] = np.select(conditions, values)
three11_small = three11_data[three11_data['Reporting Category'] != '0'].drop('SR Description', axis = 1)

In [449]:
three11_small.loc[:,'Created Date'] = pd.to_datetime(three11_small.loc[:,'Created Date'])
three11_small['year'] = three11_small.loc[:,'Created Date'].dt.to_period("Y")
#crime = crime.drop('Occurred Date Time', axis = 1)

In [456]:
three11.columns = three11_small[three11_small['year'] >= pd.Period(2011)]
three11 = three11.loc[:,('Zip Code', 'Service Request (SR) Number', 'Reporting Category', 'year')]

In [457]:
three11

,Zip Code,Service Request (SR) Number,Reporting Category,year
0,78721.0,19-00090956,animal_311,2019
2,78741.0,20-00052942,road_repair_311,2020
4,78741.0,19-00468033,request_code_officer_311,2019
7,78741.0,19-00280355,request_code_officer_311,2019
8,78745.0,19-00351755,request_code_officer_311,2019
...,...,...,...,...
1074033,78702.0,21-00184247,road_repair_311,2021
1074034,78701.0,21-00184250,loud_commercial_music_311,2021
1074045,78701.0,21-00184261,park_maintenance_311,2021
1074051,78660.0,21-00184235,animal_311,2021
